## **Procesar el Json de `fichas_tecnicas_mapped.json` para poder trabajar con Chatbots**

---
### **Librerías**

In [10]:
import json
import numpy as np
import torch
import sys
from sentence_transformers import SentenceTransformer

import matplotlib.pyplot as plt

# from sklearn.decomposition import uPCA
import seaborn as sns
import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM

---
### **1. Procesar JSON para modelo de embeddings**

---
#### **1.1 Carga de Datos**

In [11]:
# Ruta del archivo JSON (ajusta la ruta si es necesario)
json_path = "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"

# Cargar el JSON
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Mostrar los primeros elementos para revisar la estructura
print(
    json.dumps(data[:2], indent=4, ensure_ascii=False)
)  # Mostramos solo 2 para no saturar

[
    {
        "indicaciones": "en base a su efecto antiagregante plaquetario está indicado en la profilaxis de: infarto de miocardio o reinfarto de miocardio en pacientes con angina de pecho inestable y para prevenir la recurrencia del mismo en pacientes con historia de infarto de miocardio. prevención de la oclusión del bypass aortocoronario. tromboflebitis, flebotrombosis y riesgo de trombosis arteriales. tromboembolismo post-operatorio en pacientes con prótesis vasculares biológicas o con shunts arteriovenosos. tratamiento de ataques isquémicos transitorios en varones con isquemia cerebral transitoria para reducir el riesgo de accidente cerebrovascular. estas indicaciones requieren siempre un control médico.",
        "posologia": "posología: como inhibidor de la agregación plaquetaria: en la prevención de accidentes tromboembólicos (infarto, angina de pecho, prótesis valvulares biológicas, by pass arteriales, tromboflebitis, flebotrombosis y riesgo de trombosis arterial). dosis m

In [12]:
# Mostrar todas las claves de un solo medicamento para entender la estructura
print("Claves disponibles en el JSON:")
print(list(data[0].keys()))

Claves disponibles en el JSON:
['indicaciones', 'posologia', 'contraindicaciones', 'advertencias', 'interacciones', 'fertilidad_embarazo', 'efectos_conducir', 'reacciones_adversas', 'sobredosis', 'ATC', 'Propiedades_farmacocineticas', 'excipientes', 'incompatibilidades', 'precauciones_conservacion', 'fecha_revision', 'nombre_medicamento', 'ATC_Nivel_Anatomico', 'Descripcion_Nivel_Anatomico', 'ATC_Nivel_2_Subgrupo_Terapeutico', 'Descripcion_Nivel_2_Subgrupo_Terapeutico', 'ATC_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'Descripcion_Nivel_3_Subgrupo_Terapeutico_Farmacologico', 'ATC_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'Descripcion_Nivel_4_Subgrupo_Terapeutico_Farmacologico_Quimico', 'ATC_Nivel_5_Principio_Activo', 'Descripcion_Nivel_5_Principio_Activo']


---
#### **1.2 Convertir el JSON en fragmentos**

Queremos convertir el JSON inicial a una estructura tal que así:

```python
[
    {
        "medicamento": "Paracetamol",
        "categoria": "indicaciones",
        "texto": "En base a su efecto antiagregante plaquetario está indicado en..."
    },
    {
        "medicamento": "Ibuprofeno",
        "categoria": "advertencias",
        "texto": "Dado el efecto antiagregante plaquetario del ibuprofeno..."
    },
    ...
]
```

- Esta celda recorre cada medicamento del JSON y extrae los campos de interés.
- Si el valor de un campo es None, se reemplaza por una cadena vacía.
- Se elimina la extensión ".txt" del campo "nombre_medicamento" para obtener el nombre del medicamento.

In [13]:
# Generar lista de fragmentos para el chatbot

# Lista donde almacenaremos los fragmentos
fragments = []

# Definimos los campos de interés que queremos extraer
campos_interes = [
    "indicaciones",
    "posologia",
    "contraindicaciones",
    "advertencias",
    "interacciones",
    "fertilidad_embarazo",
    "efectos_conducir",
    "reacciones_adversas",
    "sobredosis",
    "Propiedades_farmacocineticas",
    "excipientes",
    "incompatibilidades",
    "precauciones_conservacion",
    "Descripcion_Nivel_Anatomico",
    "Descripcion_Nivel_2_Subgrupo_Terapeutico",
]

# Recorremos cada medicamento en el JSON
for medicamento in data:
    # Extraer el nombre del medicamento desde "nombre_medicamento" y quitar la extensión ".txt"
    nombre_medicamento = medicamento.get("nombre_medicamento", "Desconocido").replace(
        ".txt", ""
    )

    # Convertimos el nombre a minúsculas para evitar problemas de coincidencia
    nombre_medicamento = nombre_medicamento.lower()

    # Iteramos sobre cada campo de interés
    for campo in campos_interes:
        # Obtener el valor del campo; si es None, lo reemplazamos por una cadena vacía
        valor = medicamento.get(campo)
        if valor is None:
            texto = ""
        else:
            texto = valor.strip()  # Eliminamos espacios en blanco innecesarios

        # Solo añadimos el fragmento si hay contenido en el campo (no vacío)
        if texto:
            fragments.append(
                {
                    "medicamento": nombre_medicamento,  # Nombre del medicamento sin ".txt"
                    "categoria": campo,  # Categoría (por ejemplo, "indicaciones", "advertencias", etc.)
                    "texto": texto,  # Contenido del campo
                }
            )

# Mostrar algunos ejemplos de fragmentos generados para verificar la estructura
print("Ejemplo de fragmentos generados:")
print(json.dumps(fragments[:100], indent=4, ensure_ascii=False))

Ejemplo de fragmentos generados:
[
    {
        "medicamento": "a.a.s._100_mg_comprimidos",
        "categoria": "indicaciones",
        "texto": "en base a su efecto antiagregante plaquetario está indicado en la profilaxis de: infarto de miocardio o reinfarto de miocardio en pacientes con angina de pecho inestable y para prevenir la recurrencia del mismo en pacientes con historia de infarto de miocardio. prevención de la oclusión del bypass aortocoronario. tromboflebitis, flebotrombosis y riesgo de trombosis arteriales. tromboembolismo post-operatorio en pacientes con prótesis vasculares biológicas o con shunts arteriovenosos. tratamiento de ataques isquémicos transitorios en varones con isquemia cerebral transitoria para reducir el riesgo de accidente cerebrovascular. estas indicaciones requieren siempre un control médico."
    },
    {
        "medicamento": "a.a.s._100_mg_comprimidos",
        "categoria": "posologia",
        "texto": "posología: como inhibidor de la agregación p

---
### **2. Generación de embeddings**

Usamos un modelo preentrenado de **Sentence Transformers** para convertir cada fragmento en un **vector numérico (embedding)**.  

Estos embeddings capturan la **semántica** del texto, permitiendo realizar **búsquedas por significado** en lugar de solo coincidencias de palabras clave.  


In [ ]:
# Configuramos PyTorch para usar un número mayor de hilos (ajusta según tu CPU)
torch.set_num_threads(8)  # Por ejemplo, usar 8 hilos (8 cores)

# Cargamos el modelo preentrenado 'multi-qa-MiniLM-L6-dot-v1')
model = SentenceTransformer("all-MiniLM-L6-v2")    # Modelo alternativo más liviano (modelo incial)
#model = SentenceTransformer("multi-qa-MiniLM-L6-dot-v1", device="mps")
#model = SentenceTransformer("all-mpnet-base-v2", device="mps")  

# Extraemos el texto de cada fragmento
texts = [frag["texto"] for frag in fragments]

# Generamos los embeddings para cada fragmento, aumentando el batch_size para optimizar el procesamiento
embeddings = model.encode(
    texts, convert_to_numpy=True, show_progress_bar=True, batch_size=64
)

# Mostramos la dimensión de los embeddings para confirmar que se han generado correctamente
print("Número de fragmentos:", len(texts))
print("Dimensión de cada embedding:", embeddings.shape[1])

Batches:  14%|█▎        | 612/4527 [04:41<31:42,  2.06it/s] 

In [ ]:
# Guardamos el array de embeddings en un archivo .npy
output_path = "../../data/outputs/5_chatbot/embeddings_all-MiniLM-L6-v2.npy"
np.save(output_path, embeddings)

**Análisis del embedding**

In [9]:
# 1. Cargar y comprobar la forma del array de embeddings
embeddings = np.load(output_path)
print("Forma del array de embeddings:", embeddings.shape)

# 2. Mostrar las primeras 5 filas del array
print("Primeras 5 filas del array de embeddings:")
print(embeddings[:5])

# 3. Explicación sobre lo que representa cada vector de embedding
print("\nAnálisis del espacio de embeddings:")
print(
    "Cada vector de embedding es una representación numérica del texto obtenido a partir del modelo de SentenceTransformer."
)
print(
    "Cada dimensión en este vector es una característica latente aprendida durante el entrenamiento del modelo,"
)
print(
    "capaz de captar aspectos semánticos y contextuales del lenguaje. Estas dimensiones no tienen una interpretación"
)
print(
    "directa como 'la dimensión 1 representa X', sino que en conjunto forman un espacio latente que captura la semántica."
)

# Comentarios adicionales:
# - La forma del array indica el número total de documentos y la dimensión de cada embedding.
# - Cada posición en un embedding es una característica latente sin significado directo individual, pero su combinación
#   permite una representación semántica del texto.
# - La reducción a dos dimensiones mediante PCA permite identificar las direcciones que capturan mayor varianza,
#   lo cual ayuda a interpretar qué tan dispersos o agrupados están los textos en el espacio latente.
# - Técnicas adicionales como t-SNE o UMAP pueden complementarse para analizar agrupaciones semánticas en los datos.

Forma del array de embeddings: (289708, 384)
Primeras 5 filas del array de embeddings:
[[-0.07136275 -0.05918246 -0.05509703 ...  0.03421235  0.1379857
  -0.07116101]
 [ 0.0209854   0.01761444 -0.03261471 ...  0.07995179  0.03853668
  -0.04150769]
 [ 0.01360057 -0.04719002 -0.14827095 ... -0.01077812  0.08297174
   0.04240505]
 [-0.0306806  -0.04917139 -0.06687324 ...  0.003047    0.08923826
   0.02213486]
 [-0.00627262 -0.08888327 -0.07965814 ... -0.01347708  0.03768596
   0.05125593]]

Análisis del espacio de embeddings:
Cada vector de embedding es una representación numérica del texto obtenido a partir del modelo de SentenceTransformer.
Cada dimensión en este vector es una característica latente aprendida durante el entrenamiento del modelo,
capaz de captar aspectos semánticos y contextuales del lenguaje. Estas dimensiones no tienen una interpretación
directa como 'la dimensión 1 representa X', sino que en conjunto forman un espacio latente que captura la semántica.


---
### **3. Almacenar Embeddings en Base de Datos Vectorial (`FAISS`)**

Creamos un índice `FAISS` usando `IndexFlatL2`
- Un índice en `FAISS` es una estructura optimizada para almacenar y buscar embeddings.
- `IndexFlatL2` es un tipo de índice simple que usa distancia Euclidiana (L2) para comparar vectores.
- La distancia L2 (Euclidiana) entre dos vectores es la raíz cuadrada de la suma de las diferencias al cuadrado entre sus componentes.
- Se usa para medir la similitud: menor distancia = mayor similitud.



In [14]:
def build_index(index_choice):
    # Cargar los embeddings y convertir a float32
    embeddings_path = "../../data/outputs/5_chatbot/embeddings_all-MiniLM-L6-v2.npy"
    embeddings = np.load(embeddings_path).astype(np.float32)
    n, d = embeddings.shape

    # Selección del tipo de índice según el parámetro
    if index_choice == "1":
        index = faiss.IndexFlatL2(d)
        index_type = "IndexFlatL2"
    elif index_choice == "2":
        try:
            print("Creando índice IndexIVFFlat...")
            embeddings = np.load(embeddings_path).astype(np.float32)
            print(f"Dimensiones de los embeddings: {embeddings.shape}")
            n, d = embeddings.shape
            nlist = 50  # Número de listas/clusters
            quantizer = faiss.IndexFlatL2(d)
            print(f"Entrenando el cuantizador con {n} embeddings...")
            index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
            print(f"Entrenando el índice con {n} embeddings...")
            index.train(embeddings)  # Entrena el índice
            print("Entrenamiento completado.")
            index_type = "IndexIVFFlat"
        except Exception as e:
            print(f"Error al crear el índice IndexIVFFlat: {e}")
            sys.exit(1)
    elif index_choice == "3":
        index = faiss.IndexHNSWFlat(d, 32)
        index.hnsw.efConstruction = 40
        index_type = "IndexHNSWFlat"
    else:
        print("Opción no válida. Usa 1:IndexFlatL2, 2:IndexIVFFlat, 3:IndexHNSWFlat")
        sys.exit(1)

    # Agregar los embeddings al índice
    index.add(embeddings)

    # Guardar el índice en un archivo binario
    output_file = f"../../data/outputs/5_chatbot/faiss_index_{index_type}.bin"
    faiss.write_index(index, output_file)
    print(f"Índice {index_type} creado con {n} embeddings y guardado en: {output_file}")

# Elegir el tipo de índice
# 1 -> IndexFlatL2
# 2 -> IndexIVFFlat (más recomendado)
# 3 -> IndexHNSWFlat
build_index("1")

Índice IndexFlatL2 creado con 289708 embeddings y guardado en: ../../data/outputs/5_chatbot/faiss_index_IndexFlatL2.bin


In [ ]:
# !pip install faiss-cpu
# Cpu es más que suficiente para este caso, pero si se quiere usar GPU se puede instalar faiss-gpu

---
### **4. Búsqueda en `FAISS`**

Ahora que tenemos el índice FAISS creado, podemos usarlo para buscar los fragmentos más relevantes para una consulta.

**Pasos de la búsqueda**
1. **Cargamos el índice FAISS** desde el archivo guardado.
2. **Cargamos los fragmentos de texto originales** para poder recuperar la información relevante.
3. **Convertimos la consulta en un embedding** usando el mismo modelo `all-MiniLM-L6-v2`.
4. **Buscamos los embeddings más cercanos en FAISS** usando la distancia L2.
5. **Recuperamos los fragmentos de texto asociados** a los embeddings más cercanos.
6. **Devolvemos los fragmentos más relevantes** como resultados.

*Ejemplo de búsqueda**
Si la consulta del usuario es:
> "¿Cuáles son las contraindicaciones del paracetamol?"

FAISS devolverá los fragmentos más cercanos, como:
> 

Estos fragmentos se pueden usar como contexto para generar respuestas en **Llama 2** dentro de un sistema **RAG (Retrieval-Augmented Generation)**.

In [15]:
# Cargar el modelo de embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")    # old

# Cargar el índice FAISS
#index_path = "../../data/outputs/5_chatbot/faiss_index_old.bin"    # old
index_path = "../../data/outputs/5_chatbot/faiss_index_all-MiniLM-L6-v2.bin"
index = faiss.read_index(index_path)

# Cargar los datos originales (fragmentos de texto)
json_path = "../../data/outputs/2_data_preprocessing/fichas_tecnicas_mapped_atc.json"
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)
    
# Función para buscar en FAISS
def search_faiss(query, index, k=5):
    """
    Realiza una búsqueda en FAISS para encontrar los fragmentos más similares a la consulta.

    Parámetros:
    - query (str): La consulta en lenguaje natural.
    - k (int): Número de resultados a recuperar.

    Retorna:
    - Lista de fragmentos de texto relevantes.
    """
    # Convertir la consulta en embedding
    query_embedding = model.encode(query, convert_to_numpy=True).reshape(1, -1)

    # Buscar los k embeddings más cercanos
    distances, indices = index.search(query_embedding, k)

    # Recuperar los fragmentos correspondientes, incluyendo las distancias
    results = []
    for i, idx in enumerate(indices[0]):
        if idx < len(fragments):  # Asegurar que el índice es válido
            results.append(
                {
                    **fragments[idx],  # Añadir los datos del fragmento
                    "distance": distances[0][i],  # Añadir la distancia de similitud
                }
            )

    return results

# Prueba de búsqueda
query = "¿Cuáles son las reacciones adversas del paracetamol?"
resultados = search_faiss(query, index, k=10)

# Ruta donde se guardarán los resultados
output_file_path = "../../data/outputs/5_chatbot/resultados_búsqueda_FAISS.txt"

# Escribir resultados en el archivo .txt
with open(output_file_path, "w", encoding="utf-8") as f:
    for i, res in enumerate(resultados):
        f.write(f"Resultado {i+1}:\n")
        f.write(f"Medicamento: {res['medicamento']}\n")
        f.write(f"Categoría: {res['categoria']}\n")
        f.write(f"Texto: {res['texto']}\n\n")